## Solving supply-demand problem using bio-inspired algorithms

In neighborhood $Y$ of city $X$, there are 8 schools that collectively possess 100 microscopes for use in biology classes. These resources, however, are not uniformly distributed amongst the schools. With recent changes in student enrollment, 4 schools have more microscopes than needed while the other 4 schools are in need of additional ones. To address this issue, Dr. Rachel Carson, who is in charge of the biology department at City $X$'s School Board, decides to use a mathematical model. She chooses to use the Transportation Problem model, a strategy aimed at efficiently allocating resources while minimizing transportation costs [1]. The model represents supply $n$ and demand $m$ as unit weights of decision variables at various points in a network, with the cost of transporting a unit from a supply point to a demand point equivalent to the time or distance between nodes. This data is captured in an $n \times m$ cost matrix.

The formal statement of this integer linear programming problem is as follows [2]:

Minimize $\sum_{i \in I}\sum_{j \in J} c_{ij}x_{ij}$

Subject to the following constraints:

$\sum_{j \in J} x_{ij}\leq S_i  \quad \quad \forall i \in I$

$\sum_{i \in I} x_{ij}\geq D_j  \quad \quad \forall j \in J$

$x_{ij}\geq 0 \quad \quad \forall i \in I \quad \quad \forall j \in J$

where
* $i$ is each potential origin node
* $I$ is the complete set of potential origin nodes
* $j$ is each potential destination node
* $J$ is the complete set of potential nodes
* $x_{ij}$ is the amount to be shipped from $i \in I$ to $j \in J$
* $c_{ij}$ is per unit shipping costs between all $i, j$ pairs. Assume that this cost is 50*distance between between $i$ and $j$.
* $S_i$ is node i supply for $i \in I$
* $D_j$ is node i demand for $j \in J$

<b> References</b>

[1] Lovelace, R. Open source tools for geographic analysis in transport planning. Journal of Geographical Systems (2021). 

[2] Daskin, M. (2013) Network and Discrete Location: Models, Algorithms, and Applications. New York: John Wiley & Sons, Inc.

### Import used libraries

In [1]:
from pymoo.algorithms.soo.nonconvex.pso import PSO
from pymoo.optimize import minimize
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.operators.crossover.pntx import PointCrossover
from pymoo.operators.mutation.pm import PolynomialMutation
from pymoo.operators.repair.rounding import RoundingRepair
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.core.problem import Problem
import numpy as np
import matplotlib.pyplot as plt
import math
from math import exp
import random
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import folium

### Set the supply-demand problem data

In [2]:
supply_schools = [1, 6, 7, 8] # schools with microscopes available
demand_schools = [2, 3, 4, 5] # schools with microscopes requested
amount_supply = [20, 30, 15, 35] # number of microscopes available at each school
amount_demand = [5, 45, 10, 40] # number of microscopes requested at each school
n_var=len(supply_schools)*len(demand_schools) # number of variables

### Visualize the problem data

In [3]:
geolocator = Nominatim(user_agent="SupplyDemand") # create geolocator object
location = geolocator.geocode("Toronto, Ontario") # get coordinates of Toronto

In [4]:
# Function to generate random locations around a center point
def generate_random(number, center_point):
    lat, lon = center_point
    coords = [(random.uniform(lat - 0.01, lat + 0.01), random.uniform(lon - 0.01, lon + 0.01)) for _ in range(number)]
    return coords

np.random.seed(0)  # for reproducibility
supply_coords = generate_random(len(supply_schools), [location.latitude, location.longitude])
demand_coords = generate_random(len(demand_schools), [location.latitude, location.longitude])

In [5]:
# Create a map centered at Downtown Toronto
m = folium.Map(location=[location.latitude, location.longitude], zoom_start=15, scrollWheelZoom=False, dragging=False)

# Add markers for supply schools
for i, coord in zip(supply_schools, supply_coords):
    folium.Marker(location=coord, icon=folium.Icon(icon="home", color='red'), popup=f'Supply School {i+1}').add_to(m)

# Add markers for demand schools
for i, coord in zip(demand_schools, demand_coords):
    folium.Marker(location=coord, icon=folium.Icon(icon="flag", color='blue'), popup=f'Demand School {i+1}').add_to(m)

# Show the map
m

### Define Cost matrix

In [6]:
distances = []
for supply in supply_coords:
    row = []
    for demand in demand_coords:
        row.append(geodesic(supply, demand).kilometers)
    distances.append(row)

distances = np.array(distances)

cost_matrix=50*distances

### Define the transportation problem

In [7]:
class Transportation_problem(Problem):
    def __init__(self,
                 cost_matrix, 
                 amount_supply,
                 amount_demand
                 ):
        super().__init__(n_var=n_var,n_constr=1, vtype=int)
        self.cost_matrix = cost_matrix
        self.amount_supply = amount_supply
        self.amount_demand = amount_demand
        self.xl = np.array(np.zeros(n_var))
        self.xu = np.repeat(amount_supply, len(amount_demand))

    def _evaluate(self, X, out, *args, **kwargs):
        loss = np.zeros((X.shape[0], 1))
        g = np.zeros((X.shape[0], 1))
        for i in range(len(X)):
            soln = X[i].reshape(self.cost_matrix.shape)
            cost_x = X[i].reshape(self.cost_matrix.shape)
            cost = cost_x * cost_matrix.T
            cost = cost.sum()
            loss[i] = cost         
            total_supply = soln.sum(axis=1)
            total_demand = soln.sum(axis=0)  
            print("total_supply: ", total_supply)
            print("total_demand: ", total_demand)
            g[i] =  np.any(total_supply<self.amount_supply) or np.any(total_demand<self.amount_demand)
        out["F"] = loss
        out["G"] = g

In [8]:
problem = Transportation_problem(cost_matrix,amount_supply,amount_demand) # create problem object

## Define PSO solver to solve the problem

In [9]:
algorithm = PSO(pop_size=100,repair=RoundingRepair()) # create the algorithm object
res = minimize(problem, algorithm, ('n_gen', 150), seed=1, verbose=False) # run the algorithm

total_supply:  [34.57753235 43.26843817 25.51196789 64.55784944]
total_demand:  [80.58548038 26.3350346  21.68361813 39.31165474]
total_supply:  [14.92342154 50.73793755 22.96367926 64.96339617]
total_demand:  [36.84237468 26.53634193 40.77777688 49.43194103]
total_supply:  [40.71348248 68.28555958 24.54459055 68.83044934]
total_demand:  [25.5781516  17.04282887 72.16706245 87.58603903]
total_supply:  [45.91857032 71.17359909 34.84211603 41.88758602]
total_demand:  [56.16251998 43.85840156 48.2614522  45.53949772]
total_supply:  [44.50395745 60.68109695 28.94219328 82.74933521]
total_demand:  [90.44153858 57.67357635 22.07705584 46.68441213]
total_supply:  [28.47696589 81.22959404 29.88587268 77.00539443]
total_demand:  [41.14898987 68.41274863 60.1659736  46.87011494]
total_supply:  [33.28077127 60.12379755 39.70485188 53.11847562]
total_demand:  [35.41020818 49.22852005 44.00161307 57.58755501]
total_supply:  [35.73976463 55.47060065  8.29958074 71.81210376]
total_demand:  [50.800597

### Print the soltuion

In [10]:
soln = res.X

# For each supply point
for i in range(len(amount_supply)):
    print(f"Supply School({supply_schools[i]}): {' + '.join(['soln['+str(j)+']' for j in range(i*4, (i+1)*4)])} <= {amount_supply[i]} or {' + '.join(map(str, soln[i*4:(i+1)*4]))} <= {amount_supply[i]} or {sum(soln[i*4:(i+1)*4])} <= {amount_supply[i]}")

# For each demand point
for j in range(len(amount_demand)):
    print(f"Demand School({demand_schools[j]}): {' + '.join(['soln['+str(i*4+j)+']' for i in range(len(amount_demand))])} >= {amount_demand[j]} or {' + '.join(map(str, [soln[i*4+j] for i in range(len(amount_demand))]))} >= {sum(soln[i*4+j] for i in range(len(amount_demand)))} or {sum(soln[i*4+j] for i in range(len(amount_demand)))} >= {amount_demand[j]}") 

# Print shipping cost
print(f"Shipping cost =  {round(res.F[0], 2)} $")

Supply School(1): soln[0] + soln[1] + soln[2] + soln[3] <= 20 or 0 + 0 + 0 + 20 <= 20 or 20 <= 20
Supply School(6): soln[4] + soln[5] + soln[6] + soln[7] <= 30 or 0 + 8 + 10 + 12 <= 30 or 30 <= 30
Supply School(7): soln[8] + soln[9] + soln[10] + soln[11] <= 15 or 0 + 7 + 0 + 8 <= 15 or 15 <= 15
Supply School(8): soln[12] + soln[13] + soln[14] + soln[15] <= 35 or 5 + 30 + 0 + 0 <= 35 or 35 <= 35
Demand School(2): soln[0] + soln[4] + soln[8] + soln[12] >= 5 or 0 + 0 + 0 + 5 >= 5 or 5 >= 5
Demand School(3): soln[1] + soln[5] + soln[9] + soln[13] >= 45 or 0 + 8 + 7 + 30 >= 45 or 45 >= 45
Demand School(4): soln[2] + soln[6] + soln[10] + soln[14] >= 10 or 0 + 10 + 0 + 0 >= 10 or 10 >= 10
Demand School(5): soln[3] + soln[7] + soln[11] + soln[15] >= 40 or 20 + 12 + 8 + 0 >= 40 or 40 >= 40
Shipping cost =  2345.61 $


### Visualize the solution

In [11]:
# Normalize function
def normalize(lst):
    s = sum(lst)
    return list(map(lambda x: (x/s)*10, lst))

# Normalize soln array
soln_normalized = normalize(soln)

# Define a color list
colors = ['cyan', 'brown', 'orange', 'purple']

# Create a map centered at Downtown Toronto
m = folium.Map(location=[location.latitude, location.longitude], zoom_start=15, scrollWheelZoom=False, dragging=False)

# Add markers for supply schools
for i, coord in zip(supply_schools, supply_coords):
    folium.Marker(location=coord, icon=folium.Icon(icon="home", color='red'), popup=f'Supply School {i+1}').add_to(m)

# Add markers for demand schools
for i, coord in zip(demand_schools, demand_coords):
    folium.Marker(location=coord, icon=folium.Icon(icon="flag", color='blue'), popup=f'Demand School {i+1}').add_to(m)

# Add lines (edges) between supply and demand schools
for i in range(len(supply_schools)):
    for j in range(len(demand_schools)):
        soln_value = soln[i*len(demand_schools) + j]
        folium.PolyLine(locations=[supply_coords[i], demand_coords[j]], color=colors[i % len(colors)],  weight=5*soln_normalized[i*len(demand_schools) + j], popup=folium.Popup(f'# of microscopes: {soln_value}')).add_to(m)
             
# Show the map
m

## Define GA solver to solve the problem

In [12]:
algorithm = GA(
    pop_size=50,
    sampling=FloatRandomSampling(),
    crossover=PointCrossover(prob=0.8, n_points=2),
    mutation = PolynomialMutation(prob=0.3, repair=RoundingRepair()),
    eliminate_duplicates=False
) # create the algorithm object

res = minimize(problem, algorithm, ('n_gen', 600), seed=1, verbose=False) # run the algorithm

total_supply:  [28.79586891 23.12746272 30.59997335 62.31486763]
total_demand:  [25.85045767 55.993009   12.83657732 50.15812863]
total_supply:  [26.28966117 83.24258815 28.42642216 54.85995836]
total_demand:  [51.45833508 84.37573141 16.92826647 40.05629688]
total_supply:  [49.9689512  68.68677576 42.10125096 61.36652145]
total_demand:  [58.198645   62.60083409 50.39369309 50.93032719]
total_supply:  [22.32012803 30.66331439 30.14865116 56.87394454]
total_demand:  [24.29781934 27.25985942 48.27812251 40.17023685]
total_supply:  [35.29067244 77.16079248 22.64548005 96.3229585 ]
total_demand:  [63.88885738 52.59725376 72.62152653 42.31226581]
total_supply:  [52.13638149 76.76236041 35.24058675 58.57129358]
total_demand:  [51.46250572 68.91910299 43.86756207 58.46145144]
total_supply:  [41.94148496 62.9074459  37.15730615 69.91936091]
total_demand:  [65.67635684 71.5420343  32.90490595 41.80230082]
total_supply:  [49.017537   75.38089334  4.80913015 97.77991594]
total_demand:  [71.667375

### Print the solution

In [13]:
soln = res.X

# For each supply point
for i in range(len(amount_supply)):
    print(f"Supply School({supply_schools[i]}): {' + '.join(['soln['+str(j)+']' for j in range(i*4, (i+1)*4)])} <= {amount_supply[i]} or {' + '.join(map(str, soln[i*4:(i+1)*4]))} <= {amount_supply[i]} or {sum(soln[i*4:(i+1)*4])} <= {amount_supply[i]}")

# For each demand point
for j in range(len(amount_demand)):
    print(f"Demand School({demand_schools[j]}): {' + '.join(['soln['+str(i*4+j)+']' for i in range(len(amount_demand))])} >= {amount_demand[j]} or {' + '.join(map(str, [soln[i*4+j] for i in range(len(amount_demand))]))} >= {sum(soln[i*4+j] for i in range(len(amount_demand)))} or {sum(soln[i*4+j] for i in range(len(amount_demand)))} >= {amount_demand[j]}") 

# Print shipping cost
print(f"Shipping cost =  {round(res.F[0], 2)} $")

Supply School(1): soln[0] + soln[1] + soln[2] + soln[3] <= 20 or 0 + 0 + 0 + 20 <= 20 or 20 <= 20
Supply School(6): soln[4] + soln[5] + soln[6] + soln[7] <= 30 or 2 + 4 + 12 + 12 <= 30 or 30 <= 30
Supply School(7): soln[8] + soln[9] + soln[10] + soln[11] <= 15 or 1 + 13 + 0 + 3 <= 15 or 17 <= 15
Supply School(8): soln[12] + soln[13] + soln[14] + soln[15] <= 35 or 2 + 28 + 0 + 5 <= 35 or 35 <= 35
Demand School(2): soln[0] + soln[4] + soln[8] + soln[12] >= 5 or 0 + 2 + 1 + 2 >= 5 or 5 >= 5
Demand School(3): soln[1] + soln[5] + soln[9] + soln[13] >= 45 or 0 + 4 + 13 + 28 >= 45 or 45 >= 45
Demand School(4): soln[2] + soln[6] + soln[10] + soln[14] >= 10 or 0 + 12 + 0 + 0 >= 12 or 12 >= 10
Demand School(5): soln[3] + soln[7] + soln[11] + soln[15] >= 40 or 20 + 12 + 3 + 5 >= 40 or 40 >= 40
Shipping cost =  2495.1 $


### Visualize the solution

In [14]:
# Normalize function
def normalize(lst):
    s = sum(lst)
    return list(map(lambda x: (x/s)*10, lst))

# Normalize soln array
soln_normalized = normalize(soln)

# Define a color list
colors = ['cyan', 'brown', 'orange', 'purple']

# Create a map centered at Downtown Toronto
m = folium.Map(location=[location.latitude, location.longitude], zoom_start=15, scrollWheelZoom=False, dragging=False)

# Add markers for supply schools
for i, coord in zip(supply_schools, supply_coords):
    folium.Marker(location=coord, icon=folium.Icon(icon="home", color='red'), popup=f'Supply School {i+1}').add_to(m)

# Add markers for demand schools
for i, coord in zip(demand_schools, demand_coords):
    folium.Marker(location=coord, icon=folium.Icon(icon="flag", color='blue'), popup=f'Demand School {i+1}').add_to(m)

# Add lines (edges) between supply and demand schools
for i in range(len(supply_schools)):
    for j in range(len(demand_schools)):
        soln_value = soln[i*len(demand_schools) + j]
        folium.PolyLine(locations=[supply_coords[i], demand_coords[j]], color=colors[i % len(colors)],  weight=5*soln_normalized[i*len(demand_schools) + j], popup=folium.Popup(f'# of microscopes: {soln_value}')).add_to(m)
             
# Show the map
m